In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import timedelta

Вводные

Водитель может находиться в 4 возможных статусах:
    * free -- доступен для нового заказа
    * enroute -- едет на заказ
    * ontrip -- выполняет заказ
    * busy -- недоступен для нового заказа

Возможные переходы из одного состояние в другое определены как:
    * free -> [free, enroute, busy]
    * enroute -> [free, ontrip]
    * ontrip -> [free]
    * busy -> [free]

Почему переходы определяются таким образом:
1. Из состояния free можно перейти в
    * free -- если водитель ушел в офлайн и заново вышел на линию, тогда подряд будет две записи со статусом free
    * enroute -- если водитель принял заказ, то переходит в статус enroute и едет к клиенту
    * busy -- если водитель нажал кнопку "Занят" в таксометре (пошел на обед и т.д.)
2. Из состояния enroute можно перейти в 
    * free -- если клиент или водитель отменил заказ
    * ontrip -- если водитель приехал к клиенту и начал выполнять заказ
3. Из состояния ontrip можно перейти только в free (после выполнения заказа)
4. Из состояния busy можно перейти только в free

Эффективность на поездке -- это время с клиентом в машине (ontrip), деленное на сумму длительностей всех статусов, связанных с поездкой (sum(free) + enroute + ontrip), где sum(free) -- время простоя.

Время простоя -- это сумма всех статусов free, предшествующих поездке. Суммируются все статусы free, идущие подряд, а также те, которые были прерваны короткими статусами busy или enroute (короткий статус == меньше какого-то TIMEOUT'а).

Имеется набор данных со статусами водителей, по которому необходимо построить зависимость длительности поездки от эффективности.
    * driver_id -- id водителя
    * status -- один из статусов
    * dttm -- время начала статуса

Примечания:
    * Поездка считается только при наличии статуса ontrip
    * Тесты написаны для python 2

    1. Написать функцию-генератор, которая будет отдавать соседние элементы в цикле. Функция понадобится для итерирования по записям водителя и проверки соседних статусов по условиям. Не забудьте проверить, что тесты проходят без ошибок (см. test_neighbors).

In [2]:
from itertools import tee, islice, chain

def neighbors(iterable):
    # Write generator function which yields 
    # previous, current and next values in iterable list.
    # ... type your code here ...
    for i in range(len(list(iterable))):
        if i == 0:
            prv, cur, nxt = [None, iterable[0], iterable[1]]
        elif i == (len(list(iterable))-1):
            prv, cur, nxt = [iterable[i-1], iterable[i] , None]
        else:    
            prv, cur, nxt = [iterable[i-1], iterable[i] , iterable[i+1]]
        yield prv, cur, nxt

In [3]:
iterable = range(3)
for i in range(len(list(iterable))):
        
        if i == 0:
            prv, cur, nxt = [None, iterable[0], iterable[1]]
        elif i == (len(list(iterable))-1):
            prv, cur, nxt = [iterable[i-1], iterable[i] , None]
        else:    
            prv, cur, nxt = [iterable[i-1], iterable[i] , iterable[i+1]]
        print (prv,cur, nxt)

(None, 0, 1)
(0, 1, 2)
(1, 2, None)


In [4]:
for i in neighbors(iterable):
    print(i)

(None, 0, 1)
(0, 1, 2)
(1, 2, None)


In [5]:
# Check if test passes
def test_neighbors():
    test_neighbors = neighbors( range(2) )
    assert test_neighbors.next() == (None, 0, 1)

test_neighbors()

    2. Сгруппировать данные на уровне водителя таким образом, чтобы в одной строке находились все его записи со статусами и началом статуса списком:

Формат исходной таблицы:
<table>
<tr><td>driver_id</td><td>status</td><td>dttm</td></tr>
<tr><td>9f8f9bf3ee8f4874873288c246bd2d05</td><td>free</td><td>2018-02-04 00:19</td></tr>
<tr><td>9f8f9bf3ee8f4874873288c246bd2d05</td><td>busy</td><td>2018-02-04 01:03</td></tr>
<tr><td>8f174ffd446c456eaf3cca0915d0368d</td><td>free</td><td>2018-02-03 15:43</td></tr>
<tr><td>8f174ffd446c456eaf3cca0915d0368d</td><td>enroute</td><td>2018-02-03 17:02</td></tr>
<tr><td>...</td><td>...</td><td>...</td></tr>
</table>

Формат сгруппированной таблицы:
<table>
<tr><td>driver_id</td><td>driver_info</td></tr>
<tr><td>9f8f9bf3ee8f4874873288c246bd2d05</td><td>[("free", 2018-02-04 00:19), ("busy", 2018-02-04 01:03)]</td></tr>
<tr><td>8f174ffd446c456eaf3cca0915d0368d</td><td>[("free", 2018-02-03 15:43), ("enroute", 2018-02-03 17:02) ...]</td></tr>
</table>

In [6]:
df = pd.read_csv("dataset.csv", parse_dates=["dttm"])
drivers_list = df.driver_id.unique()

In [7]:
big_list = []
for i in drivers_list:
  
    df1 = df[df['driver_id'] == i]
    
    list1 = []
    for j in df1.values:
        list1.append((j[0], j[2]))
    big_list.append([i, list1])

In [8]:
df_grouped = pd.DataFrame(big_list, columns=['driver_id', 'driver_info'])

In [9]:
df_grouped.head()

,driver_id,driver_info
0,8f174ffd446c456eaf3cca0915d0368d,"[(free, 2018-02-18 20:51:22.620339), (enroute,..."
1,dc66190e523943f5a83bdd393587439c,"[(free, 2018-02-18 20:02:47.620339), (enroute,..."
2,17833e71aa58494ea0cb0a53be840c4e,"[(free, 2018-02-18 20:19:26.620339), (enroute,..."
3,63f6aaaf48844d1e83e9fc78716c601e,"[(free, 2018-02-18 20:52:11.620339), (enroute,..."
4,78938964fed443cf85879cbf71c6ea56,"[(free, 2018-02-18 20:08:57.620339), (busy, 20..."


    3. Используя функцию neighbors, написать функцию, которая для каждой записи в списке driver_info посчитает ее длительность.

In [10]:
def calc_status_duration(driver_info):
    driver_info_updated = []
    for i, j, k in neighbors(driver_info):
        if k == None:
            delta = None 
        else:
            delta = (k[1]- j[1])
            delta = delta.seconds
              
        driver_info_updated.append ( (j[0], j[1], delta))
        
    return driver_info_updated

In [11]:
sample_driver_info = [("free", datetime(2018, 4, 2, 0, 19)), 
                      ("busy", datetime(2018, 4, 2, 1, 3)),
                      ("busy", datetime(2018, 4, 2, 1, 4)), ]

In [12]:
for i, j, k in neighbors(sample_driver_info):
    print(i,j,k)

(None, ('free', datetime.datetime(2018, 4, 2, 0, 19)), ('busy', datetime.datetime(2018, 4, 2, 1, 3)))
(('free', datetime.datetime(2018, 4, 2, 0, 19)), ('busy', datetime.datetime(2018, 4, 2, 1, 3)), ('busy', datetime.datetime(2018, 4, 2, 1, 4)))
(('busy', datetime.datetime(2018, 4, 2, 1, 3)), ('busy', datetime.datetime(2018, 4, 2, 1, 4)), None)


In [13]:
calc_status_duration(sample_driver_info)

[('free', datetime.datetime(2018, 4, 2, 0, 19), 2640),
 ('busy', datetime.datetime(2018, 4, 2, 1, 3), 60),
 ('busy', datetime.datetime(2018, 4, 2, 1, 4), None)]

In [14]:
# Check if test passes
def test_calc_status_duration():
    sample_driver_info = [("free", datetime(2018, 4, 2, 0, 19)), 
                          ("busy", datetime(2018, 4, 2, 1, 3)),]
    sample_driver_info_updated = [('free', datetime(2018, 4, 2, 0, 19), 2640.0),
                                  ('busy', datetime(2018, 4, 2, 1, 3), None),]
    assert calc_status_duration(sample_driver_info) == sample_driver_info_updated

test_calc_status_duration()

In [15]:
df_grouped["driver_info"] = df_grouped.driver_info.apply(calc_status_duration)

In [19]:
df_grouped.driver_info[0]

[('free', Timestamp('2018-02-18 20:51:22.620339'), 1470),
 ('enroute', Timestamp('2018-02-18 21:15:52.620339'), 5596),
 ('free', Timestamp('2018-02-18 22:49:08.620339'), 1289),
 ('busy', Timestamp('2018-02-18 23:10:37.620339'), 5306),
 ('free', Timestamp('2018-02-19 00:39:03.620339'), 1859),
 ('enroute', Timestamp('2018-02-19 01:10:02.620339'), 362),
 ('free', Timestamp('2018-02-19 01:16:04.620339'), 1499),
 ('busy', Timestamp('2018-02-19 01:41:03.620339'), 1213),
 ('free', Timestamp('2018-02-19 02:01:16.620339'), 5189),
 ('enroute', Timestamp('2018-02-19 03:27:45.620339'), 2199),
 ('free', Timestamp('2018-02-19 04:04:24.620339'), 1350),
 ('enroute', Timestamp('2018-02-19 04:26:54.620339'), 5515),
 ('ontrip', Timestamp('2018-02-19 05:58:49.620339'), 1664),
 ('free', Timestamp('2018-02-19 06:26:33.620339'), 96),
 ('busy', Timestamp('2018-02-19 06:28:09.620339'), 1123),
 ('free', Timestamp('2018-02-19 06:46:52.620339'), 4409),
 ('busy', Timestamp('2018-02-19 08:00:21.620339'), 1011),
 ('

    4. Используя функцию neighbors, написать функцию, которая сформирует из списка driver_info список поездок с информацией о длительности поездки и эффективности (duration_ontrip, efficiency).

In [21]:
def ifnull(var, val):
      if var is None:
        return val
      return var

In [167]:
TIMEOUT = 1600

def collapse_statuses(driver_info):
    count_free = []
    count_ontrip = []
    count_enroute = []

    for i, j, k in neighbors(driver_info):
    
        a = ("", 0, 0.)
        i = ifnull(i, a)
        k = ifnull(k, a)
        j = ifnull(j, a)
        
        if i[0]== "free" and k[0]=="free" and j[2] <= TIMEOUT:
       
            count_free.append(i[2])
            count_free.append(k[2])

        elif i[0] == "free" and k[0]!="free" and j[2]> TIMEOUT:

            count_free.append(ifnull(i[2],0))
        elif i[0] == "ontrip":
            count_ontrip.append(i[2])
        elif i[0] == "enroute" and j[0]!="free":
            count_enroute.append(i[2])

    z = [(sum(count_ontrip), sum(count_ontrip)/(sum(count_free)+sum(count_enroute)+sum(count_ontrip)))]
    return z

In [168]:
sample_driver_info = [("free", datetime(2018, 4, 2, 0, 19), 2640.0), 
                          ("busy", datetime(2018, 4, 2, 1, 3), 1660.0),
                          ("free", datetime(2018, 4, 2, 1, 30, 40), 2050.0),
                          ("enroute", datetime(2018, 4, 2, 2, 4, 50), 70.0),
                          ("free", datetime(2018, 4, 2, 2, 6), 500.0),
                          ("enroute", datetime(2018, 4, 2, 2, 14, 20), 520.0),
                          ("ontrip", datetime(2018, 4, 2, 2, 23), 3060.0),
                          ("free", datetime(2018, 4, 2, 3, 14), None)
                         ]

In [169]:
count_free = []
count_ontrip = []
count_enroute = []

for i, j, k in neighbors(sample_driver_info):
    
    a = ("", 0, 0.)
    i = ifnull(i, a)
    k = ifnull(k, a)
    j = ifnull(j, a)
    print(i[0], i[2], j[0],  j[2], k[0], k[2])
    if i[0]== "free" and k[0]=="free" and j[2] <= TIMEOUT:
        print(1)
        count_free.append(i[2])
        count_free.append(k[2])
        print(i[2],k[2])
    elif i[0] == "free" and k[0]!="free" and j[2]> TIMEOUT:
        print(2)
        count_free.append(ifnull(i[2],0))
    elif i[0] == "ontrip":
        count_ontrip.append(i[2])
    elif i[0] == "enroute" and j[0]!="free":
        count_enroute.append(i[2])
print(count_free)
z = [(sum(count_ontrip), sum(count_ontrip)/(sum(count_free)+sum(count_enroute)+sum(count_ontrip)))]

('', 0.0, 'free', 2640.0, 'busy', 1660.0)
('free', 2640.0, 'busy', 1660.0, 'free', 2050.0)
('busy', 1660.0, 'free', 2050.0, 'enroute', 70.0)
('free', 2050.0, 'enroute', 70.0, 'free', 500.0)
1
(2050.0, 500.0)
('enroute', 70.0, 'free', 500.0, 'enroute', 520.0)
('free', 500.0, 'enroute', 520.0, 'ontrip', 3060.0)
('enroute', 520.0, 'ontrip', 3060.0, 'free', None)
('ontrip', 3060.0, 'free', None, '', 0.0)
[2050.0, 500.0]


In [170]:
z

[(3060.0, 0.499184339314845)]

In [171]:
# Check if test passes
def test_collapse_statuses():
    sample_driver_info = [("free", datetime(2018, 4, 2, 0, 19), 2640.0), 
                          ("busy", datetime(2018, 4, 2, 1, 3), 1660.0),
                          ("free", datetime(2018, 4, 2, 1, 30, 40), 2050.0),
                          ("enroute", datetime(2018, 4, 2, 2, 4, 50), 70.0),
                          ("free", datetime(2018, 4, 2, 2, 6), 500.0),
                          ("enroute", datetime(2018, 4, 2, 2, 14, 20), 520.0),
                          ("ontrip", datetime(2018, 4, 2, 2, 23), 3060.0),
                          ("free", datetime(2018, 4, 2, 3, 14), None)
                         ]
    sample_driver_info_updated = [(3060.0, 3060.0 / (3060.0 + 520.0 + 500.0 + 2050.0))]
    assert collapse_statuses(sample_driver_info) == sample_driver_info_updated

test_collapse_statuses()

In [172]:
df_grouped["driver_info"] = df_grouped.driver_info.apply(collapse_statuses)

IndexError: list index out of range

In [173]:
df_grouped.head()

,driver_id,driver_info
0,8f174ffd446c456eaf3cca0915d0368d,"[(3060.0, 0.499184339315)]"
1,dc66190e523943f5a83bdd393587439c,"[(3060.0, 0.499184339315)]"
2,17833e71aa58494ea0cb0a53be840c4e,"[(3060.0, 0.499184339315)]"
3,63f6aaaf48844d1e83e9fc78716c601e,"[(3060.0, 0.499184339315)]"
4,78938964fed443cf85879cbf71c6ea56,"[(3060.0, 0.499184339315)]"


    5. Нарисовать и проинтерпретировать зависимость между длительностью поездки и эффективностью.
    Подсказка: требуется сделать обратное преобразование из таблицы со строками на уровне водителя в таблицу со строками на уровне поездки.